#### Vader_lexicon 파일에 한글로 점수를 부여한 결과 점수가 Compound되서 나온 것을 알 수 있었음 . 따라서 trans를 사용하여 lexicon file을 한글로 번역한후 저장해봄. 
#### 이후 진행사항 => 논문처럼 뉴스를 시간대 별로 크롤링, TF(TF 이후 TF-IDF)를 이용하여 감성 사전을 구축한다. 이를 위해서는 등락률 또한 필요함.
#### YG OR KOSPI 종목을 선택하여 진행


In [27]:
from googletrans import Translator
translator = Translator()

In [26]:
with open('vader_lexicon_kr.txt', 'r') as f:
    line = f.readlines()
    print(line)

['$:\t-1.5\t0.80623\t[-1, -1, -1, -1, -3, -1, -3, -1, -2, -1]\n', '%)\t-0.4\t1.0198\t[-1, 0, -1, 0, 0, -2, -1, 2, -1, 0]\n', '%-)\t-1.5\t1.43178\t[-2, 0, -2, -2, -1, 2, -2, -3, -2, -3]\n', '&-:\t-0.4\t1.42829\t[-3, -1, 0, 0, -1, -1, -1, 2, -1, 2]\n', '&:\t-0.7\t0.64031\t[0, -1, -1, -1, 1, -1, -1, -1, -1, -1]\n', "( '}{' )\t1.6\t0.66332\t[1, 2, 2, 1, 1, 2, 2, 1, 3, 1]\n", '(%\t-0.9\t0.9434\t[0, 0, 1, -1, -1, -1, -2, -2, -1, -2]\n', "('-:\t2.2\t1.16619\t[4, 1, 4, 3, 1, 2, 3, 1, 2, 1]\n", "(':\t2.3\t0.9\t[1, 3, 3, 2, 2, 4, 2, 3, 1, 2]\n", '((-:\t2.1\t0.53852\t[2, 2, 2, 1, 2, 3, 2, 2, 3, 2]\n', '(*\t1.1\t1.13578\t[2, 1, 1, -1, 1, 2, 2, -1, 2, 2]\n', '(-%\t-0.7\t1.26886\t[-1, 2, 0, -1, -1, -2, 0, 0, -3, -1]\n', '(-*\t1.3\t1.26886\t[4, 1, 2, 0, 2, -1, 1, 2, 1, 1]\n', '(-:\t1.6\t0.8\t[2, 2, 1, 3, 1, 1, 1, 3, 1, 1]\n', '(-:0\t2.8\t0.87178\t[3, 2, 3, 4, 3, 2, 3, 1, 4, 3]\n', '(-:<\t-0.4\t2.15407\t[-3, 3, -1, -1, 2, -1, -2, 3, -3, -1]\n', '(-:o\t1.5\t0.67082\t[3, 1, 1, 2, 2, 2, 1, 1, 1, 1]\n', '

In [51]:
words = [] 
for i in range(len(line)):
    a = line[i]
    word = a.split('\t')[0]
    words.append(word)
print(len(words))

scores = [] 
for i in range(len(line)):
    a = line[i]
    score = a.split('\t')[1]
    scores.append(score)
print(len(scores))

7520
7520


In [70]:
source = {'words' : words, 'scores' : scores}

In [74]:
df = pd.DataFrame(source)
df

,words,scores
0,$:,-1.5
1,%),-0.4
2,%-),-1.5
3,&-:,-0.4
4,&:,-0.7
...,...,...
7515,}:,-2.1
7516,}:(,-2.0
7517,}:),0.4
7518,}:-(,-2.1


In [78]:
re = translator.translate(words, dest='ko', src='auto')

KeyboardInterrupt: 

In [38]:
re.text

'아름답게 2.6 0.8 [2, 3, 3, 3, 2, 1, 3, 4, 2, 3]'

#### 위에 번역작업이 오래걸려 word file 을 500개 씩 나누어 저장한뒤 PAPAGO로 직접 번역후 저장 해본다.

In [77]:
with open('words.txt', 'w') as f:
    for line in words:
        f.write(line+'\n')

In [85]:
a = 0
b = 500 
print(a*b, (a+1)*b)

0 500


In [114]:
# 0~ 7500 까지 word를 저장 word_1 ~ word_14 까지 저장이 된다. 
b = 500
for i in range(len(words)//500):
    with open(f'./vader_lexicon_en/words_{i}.txt', 'w') as f:
        for line in words[int(i*b): int((i+1)*b)]:
            f.write(line+'\n')

In [141]:
# 7500~ 마지막 words 까지 저장 .
with open('./vader_lexicon_en/words_15.txt', 'w') as f:
    for line in words[7500:len(words)]:
        f.write(line+'\n')

#### 이후 수작업으로 PAPAGO 번역기를 이용하여 word를 번역해본다. 

#### 번역과정에서 불필요한 . 을 확인하고 모두 삭제해줌.

In [152]:
# 번역이 완료된 파일을 불러와 본다 .
words_kr = []
for i in range(16) :
    with open(f'./vader_lexicon_kr/words_{i}.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            a = line.replace('\n', '')
            words_kr.append(a)

In [154]:
# 번역이 완료된 7520개의 한글 words를 얻을 수 있음.
print(len(words_kr))

7520


In [156]:
# 이를 word_kr score의 형태로 만들어 준다 .
source = {'words_kr' : words_kr, 'scores' : scores}
df = pd.DataFrame(source)
df

,words_kr,scores
0,$:,-1.5
1,%),-0.4
2,%-),-1.5
3,&-:,-0.4
4,&:,-0.7
...,...,...
7515,}:,-2.1
7516,}:(,-2.0
7517,}:),0.4
7518,}:-(,-2.1


In [159]:
# 데이터 프레임을 txt 형태로 저장해준다. 
df.to_csv('vader_lexicon_kr.txt', index=False, header=None, sep="\t")

#### 이후 anaconda site package의 vaderSentiment.py 파일안에 vader_lexicon 의 경로를 vader_lexicon_kr.txt로 재설정 해준다.
#### 끝